In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
sys.path.append('/home/Public/SiameseEQTransformer/S-EqT_open_source/SiameseEarthquakeTransformer/S_EqT_codes/')
import warnings
warnings.filterwarnings("ignore")
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
from keras.models import Model
import numpy as np
from src.S_EqT_concate_fix_corr import S_EqT_Concate_RSRN_Model, feature_map_corr_func, feature_map_corr_layer, build_corr_model
from src.data_preprocessing import get_siamese_input_list, get_ori_response_list
import yaml
from src.data_preprocessing import normalize_by_std
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Explore hidden responses in the P branch

### Load EqT and S-EqT models

In [2]:
cfgs = yaml.load(open('/home/Public/SiameseEQTransformer/S-EqT_open_source/SiameseEarthquakeTransformer/S_EqT_codes/configs/P_branch.yaml','r'), Loader=yaml.BaseLoader)
encode_model, siamese_model, EqT_model = S_EqT_Concate_RSRN_Model(cfgs)
siamese_model.load_weights('/home/Public/SiameseEQTransformer/S-EqT_open_source/SiameseEarthquakeTransformer/models/S_EqT/S_EqT_P_branch.hdf5')
SEqT_penultimate = Model(siamese_model.input,siamese_model.get_layer('concatenate_26').output)
model_corr = build_corr_model(cfgs = cfgs, EqT_model = EqT_model)

Start loading EqT model...


Start building EqT encoder model...
Start building Siamese EqT model...
Start building correlation layers...
Start sideoutput & residual layers...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 6000, 3)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 6000, 8)      272         input[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 3000, 8)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1

### Load example data

In [3]:
four_examples = np.load('/home/Public/SiameseEQTransformer/S-EqT_open_source/SiameseEarthquakeTransformer/Tutorial/01_explore_hidden_responses_in_the_pretrained_EqT/Four_examples.npy',allow_pickle=True)[()]
template = four_examples['CI.DEC']
search = four_examples['CI.RIN']

### Calculate hidden responses inside the EqT and S-EqT models

In [4]:
# get hidden responses and predictions from the pre-trained EqT model
# template data
data_t = template['data']
data_t = normalize_by_std(data_t)
data_t_in = np.zeros([1,6000,3])
data_t_in[0,:,:] = data_t[:,:]
encoded_t = encode_model.predict(data_t_in)
template_eqt_pred = EqT_model.predict(data_t_in)

# get EqT prediction on the templates
template_eqt_output = EqT_model.predict(data_t_in)
spt_t_eqt = np.argmax(template_eqt_output[1][0,:,0])
sst_t_eqt = np.argmax(template_eqt_output[2][0,:,0])

# search data
data_s = search['data']
data_s = normalize_by_std(data_s)
data_s_in = np.zeros([1,6000,3])
data_s_in[0,:,:] = data_s[:,:]
encoded_s = encode_model.predict(data_s_in)
search_eqt_pred = EqT_model.predict(data_s_in)

In [5]:
# origion responses and their cross-corelations
#ori_response_list = get_ori_response_list(cfgs, spt_t_eqt, sst_t_eqt, encoded_t, encoded_s)
#ori_corr = model_corr.predict(ori_response_list)
# enhanced responses and their cross-corelations

#enhanced_corr =  model_corr.predict(siamese_input_list)
siamese_input_list = get_siamese_input_list(cfgs, spt_t_eqt, sst_t_eqt, encoded_t, encoded_s)
search_seqt_pred = siamese_model.predict(siamese_input_list)